In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
#import keras

import statistics

from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_curve, auc
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, losses
from tensorflow.keras.models import Model

In [ ]:
tf.__version__
from platform import python_version
print(python_version())

In [2]:
def my_mode( input_data ):

    freq_count = dict() 

    for d in input_data: 
        if ( d in freq_count ):
            freq_count[d] += 1 
        else: 
            freq_count[d] = 1 

    max_count = 0 
    mode_value = None 

    for k,v in freq_count.items():
        if ( v > max_count ):
            mode_value = k 
            max_count = v

    return mode_value ; 

In [29]:
def normalise_data(input_data):

    #print(input_data[0:100])
    max_value = np.argmax(input_data)
    min_value = np.argmin(input_data)
    for i in range(0, len(input_data)):
        input_data[i] = (input_data[i] - min_value)/(max_value-min_value)
    return input_data

In [35]:
def generate_data_samples( input_dataset, x_columns, y_column, sample_size, y_compress_function=my_mode):

    y_values = [] 

    # First take the y series data and apply the compression function to get a single value per sample. 
    y_series = input_dataset[y_column]; 

    for i in range(0,len(input_dataset), sample_size): 

        if ( (i + 128) <= len(y_series)):
            value = y_compress_function(y_series[i:i+sample_size])
            y_values.append(value)
            

    # Now truncate the dataset, since we need the rows to be divisible by the sample size. 
    input_dataset = input_dataset.truncate(after=len(y_values*sample_size)-1)

    input_dataset = input_dataset[x_columns]
    input_dataset.replace(np.nan, 0)
    np_x_values = input_dataset.to_numpy()
    np_x_values = np_x_values.astype(np.float32)
    
    #Regularize the data 
    np_x_values.shape
    for i in range (0, np_x_values.shape[1]): 
        normalise_data(np_x_values[:,i])     
    np_x_values.shape
    np_x_values = np_x_values.reshape(-1, sample_size, np_x_values.shape[1] )
    np_x_values = np_x_values.reshape(np_x_values.shape[0], np_x_values.shape[1], np_x_values.shape[2], 1) 
    
    np_y_values = np.array(y_values); 
    np_y_values = np_y_values.reshape(np_y_values.shape[0], 1)

    return np_x_values.astype(np.float32), np_y_values.astype(np.int32) 

In [11]:

traindata_day1 = pd.read_csv("./3_BOS_LeftLowerLimb/rawData_Day1_l.csv")
#traindata_day1.head()
traindata_day4 = pd.read_csv("./3_BOS_LeftLowerLimb/rawData_Day4_l.csv")
#traindata_day4.head()

C:\Users\svetl\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (2,3,4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Merge Patient 10_BOS data for days 1 and 4 to create a single dataframe that will be used for validation.

In [36]:
trainingdata = pd.concat([traindata_day1, traindata_day4], ignore_index=True)
X_train, Y_train = generate_data_samples(trainingdata, ['leftAnkle_X', 'leftAnkle_Y','leftAnkle_Z'], 'Dysk_Score', 128)
#print(X_train[0:10])
#print(Y_train[0:10])
print(len(trainingdata))
#training_data = trainingdata[ ['leftAnkle_X', 'leftAnkle_Y','leftAnkle_Z'] ] #create a dataframe for 3 axel readings
#training_labels = trainingdata['Dysk_Score'] # create a dataframe for the labels
#print(len(training_labels))

[-0.30005 -0.01596 -0.14405 -0.01809 -0.32538 -0.12244 -0.18662 -0.10299
 -0.03336 -0.15363 -0.20641 -0.28103 -0.17679 -0.31298 -0.09842 -0.20488
 -0.09749 -0.27295 -0.21574 -0.24924 -0.17514 -0.02196 -0.21193 -0.12727
 -0.17258 -0.15161 -0.14614 -0.26023 -0.25426  0.01169 -0.13273 -0.14219
 -0.29519 -0.19437 -0.14852 -0.14671 -0.30756 -0.0338  -0.12935 -0.17719
  0.00575 -0.15944 -0.15554 -0.1585  -0.15541 -0.1604  -0.14627  0.01069
 -0.21119 -0.28041 -0.1633  -0.10771 -0.03111 -0.16023 -0.21065 -0.27878
 -0.14868 -0.09378 -0.03699 -0.17882 -0.16607 -0.07461 -0.09938 -0.41613
 -0.0966   0.00106  0.0266  -0.12087 -0.05813 -0.0754  -0.19863 -0.1264
 -0.19187 -0.01346 -0.11302 -0.18965 -0.00946 -0.11697 -0.31359 -0.17391
 -0.15066 -0.16317 -0.0042  -0.15053 -0.16506 -0.14127 -0.01272 -0.14971
  0.00295 -0.18557 -0.12479  0.00946 -0.01262 -0.22698 -0.27684 -0.17312
 -0.3261  -0.0423  -0.06494 -0.16015]
[-0.15265 -0.16153 -0.15065 -0.1685  -0.13273 -0.02342 -0.34851 -0.27445
 -0.14706 -0.1

In [38]:
from sklearn.utils import class_weight
print(np.unique(Y_train))
class_weights = class_weight.compute_class_weight('balanced',
                                                 np.unique(Y_train),
                                                 Y_train)

[0 1]


TypeError: unhashable type: 'numpy.ndarray'

In [41]:
valdata_day1 = pd.read_csv("./17_BOS_LeftLowerLimb/rawData_Day1_l.csv")
valdata_day4 = pd.read_csv("./17_BOS_LeftLowerLimb/rawData_Day4_l.csv")
valdata_day1.head()

,Unnamed: 0,timestamp,leftAnkle_X,leftAnkle_Y,leftAnkle_Z,leftAnkle_Magnitude,Dysk_Score
0,0,1.444037e+09,0.15686,0,9.72549,9.72675,0
1,1,1.444037e+09,0.31372,0,9.56863,9.57377,0
2,2,1.444037e+09,0.15686,-0.15686,9.88235,9.88484,0
3,3,1.444037e+09,0.47059,-0.15686,10.0392,10.0515,0
4,4,1.444037e+09,0.15686,0.15686,9.56863,9.5712,0


Merge Patient 17_BOS data for days 1 and 4 to create a single dataframe that will be used for validation.

In [42]:
valdata = pd.concat([valdata_day1, valdata_day4], ignore_index=True)
X_val, Y_val = generate_data_samples(valdata, ['leftAnkle_X', 'leftAnkle_Y','leftAnkle_Z'], 'Dysk_Score', 128)
#print(X)
#print(Y)

print(len(valdata))
print(X_val[0:10])

[0.15686 0.31372 0.15686 0.47059 0.15686 0.47059 0.31372 0.31373 0.31372
 0.31372 0.31372 0.31372 0.15686 0.15686 0.15686 0.47059 0.31372 0.15686
 0.15686 0.31373 0.31372 0.31373 0.47059 0.15686 0.47059 0.31372 0.15686
 0.47059 0.31372 0.15686 0.31372 0.15686 0.31373 0.31372 0.15686 0.31372
 0.31372 0.30754 0.15717 0.32099 0.30671 0.15299 0.157   0.16022 0.32485
 0.46975 0.30362 0.15107 0.16781 0.48195 0.45892 0.14283 0.17997 0.62299
 0.14476 0.32417 0.3064  0.15105 0.16508 0.31133 0.15594 0.32209 0.30355
 0.16457 0.47687 0.30096 0.15462 0.15623 0.16166 0.31984 0.30683 0.15364
 0.15425 0.17057 0.46768 0.15021 0.32316 0.30499 0.15774 0.32127 0.30504
 0.15941 0.31448 0.14901 0.16518 0.31298 0.14924 0.16573 0.31052 0.15854
 0.31251 0.15777 0.31357 0.1543  0.32643 0.46935 0.30367 0.15129 0.16687
 0.3039 ]
[ 0.0000e+00  0.0000e+00 -1.5686e-01 -1.5686e-01  1.5686e-01  0.0000e+00
  1.5686e-01 -1.5686e-01  0.0000e+00  0.0000e+00  0.0000e+00  1.5686e-01
 -3.1372e-01  0.0000e+00  0.0000e+00 -1.5

In [48]:
testdata_day1 = pd.read_csv("./11_BOS_LeftLowerLimb/rawData_Day1_l.csv")
testdata_day4 = pd.read_csv("./11_BOS_LeftLowerLimb/rawData_Day4_l.csv")
testdata_day1.head()

,Unnamed: 0,timestamp,leftAnkle_X,leftAnkle_Y,leftAnkle_Z,leftAnkle_Magnitude,Dysk_Score
0,0,1.430815e+09,0,-0.47059,10.1961,10.2069,0
1,1,1.430815e+09,-0.15686,-0.15686,10.1961,10.1985,0
2,2,1.430815e+09,-0,-0.31372,10.1961,10.2009,0
3,3,1.430815e+09,-0.15686,-0.15686,10.1961,10.1985,0
4,4,1.430815e+09,-0.31372,-0.15686,10.1961,10.2021,0


In [49]:
testdata = pd.concat([testdata_day1, testdata_day4], ignore_index=True)
X_test, Y_test = generate_data_samples(testdata, ['leftAnkle_X', 'leftAnkle_Y','leftAnkle_Z'], 'Dysk_Score', 128)
#print(X)
#print(Y)
print(len(testdata))
print(X_test[0:10])

[ 0.0000e+00 -1.5686e-01 -0.0000e+00 -1.5686e-01 -3.1372e-01 -3.1373e-01
 -1.5687e-01 -1.5686e-01 -1.5686e-01 -1.5686e-01 -1.5686e-01 -3.1372e-01
 -1.5688e-01 -1.0000e-05  2.0000e-05 -3.1373e-01 -1.0000e-05 -1.5685e-01
 -1.5686e-01 -3.1372e-01 -1.5688e-01 -1.5686e-01 -1.5685e-01 -3.1372e-01
 -1.5690e-01  3.0000e-05 -3.1373e-01 -0.0000e+00 -3.1370e-01 -1.5689e-01
 -1.5684e-01 -3.1372e-01 -1.5690e-01 -2.8000e-04 -1.6851e-01 -3.1371e-01
 -1.4299e-01 -8.6800e-03 -3.1971e-01 -1.4424e-01  2.3600e-03  2.4000e-04
 -3.3500e-03 -1.6828e-01 -3.1153e-01 -1.5197e-01 -1.5637e-01 -1.6374e-01
 -3.1122e-01 -1.5775e-01 -3.1493e-01 -1.4890e-01 -1.6529e-01 -3.1249e-01
 -1.5111e-01 -1.5885e-01 -1.5465e-01 -1.6375e-01 -3.1289e-01 -1.5104e-01
 -1.5875e-01 -1.5512e-01 -1.6196e-01 -3.1956e-01 -3.0749e-01 -1.5147e-01
 -1.6246e-01 -3.2121e-01 -3.0037e-01  2.9900e-03 -1.6222e-01 -1.6064e-01
 -3.1773e-01 -3.1614e-01 -3.0003e-01 -3.4600e-03 -3.1806e-01 -1.5604e-01
 -3.1493e-01 -1.5060e-01 -1.5853e-01 -1.5645e-01 -1

In [ ]:
#testdata.shape
print(X_test[0][0])

Set up hyperparameters

In [43]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Flatten, Conv1D, Conv2D, MaxPooling1D, MaxPooling2D
from tensorflow.keras.optimizers import Adam, schedules, SGD

From TinyML book: Very simple model to prototype
This is a sequential model, meaning the output of each layer is passed directly into the next one.
The first layer is Conv2D: This is a convolutional layer; it directly receives the network’s input, which is a sequence of raw accelerometer data. The input’s shape is provided in the input_shape argument. It’s set to (seq_length, 3, 1), where seq_length is the total number of accelerometer measurements that are passed in (128 by default). Each measurement is composed of three values, representing the x-, y-, and z-axes. 
The role of the convolutional layer is to take the raw data and extract some basic features that can be interpreted by subsequent layers. The arguments to the Conv2D() function determine how many features will be extracted.
The first argument determines how many filters the layer will have. During training, each filter learns to identify a particular feature in the raw data—for example, one filter might learn to identify the signs of an upward motion. For each filter, the layer outputs a feature map that shows where the feature it has learned occurs within the input.
The layer defined in the code has 8 filters, meaning that it will learn to recognize and output eight different types of high-level features from the input data. This is reflected in the output shape, (batch_size, 128, 3, 8), which has eight feature channels in its final dimension, one for each feature. The value in each channel indicates the degree to which a feature was present in that location of the input.
The convolutional layers slide a window across the data and decide whether a given feature is present in that window. The second argument to Conv2D() is where we provide the dimensions of this window. In our case, it’s (4, 3). This means that the features for which our filters are hunting span four consecutive accelerometer measurements and all three axes. Because the window spans four measurements, each filter analyzes a small snapshot of time, meaning it can generate features that represent a change in acceleration over time. 
When used for classification, the goal of a CNN is to transform a big, complex input tensor into a small, simple output. This is what the MaxPool2D layer does by congesting the output of the first convolutional layer into a concentrated, high-level representation of the most relevant features it contains. Even though the original input has 3 accelerometer axes for each measurement, a combination of Conv2D and MaxPool2D merges these together into a single value.
The Dropout layer randomly sets some of a tensor’s values to zero during training. In this case, by calling Dropout(0.1), we set 10% of the values to zero, entirely obliterating that data. Dropout is a regularization technique and process of improving machine learning models so that they are less likely to overfit their training data. By randomly removing some data between one layer and the next, we force the neural network to learn how to cope with unexpected noise and variation.
There can be more Conv2D and MaxPooling layers; let us strat with 2.
The Flatten layer is used to transform a multidimensional tensor into one with a single dimension. In this case, our (14, 1, 16) tensor is squished down into a single dimension with shape (224).
It’s then fed into a Dense layer with 16 neurons, which considersg all data at once, learning the meanings of various combinations of inputs. The output of this Dense layer will be a set of 16 values representing the content of the original input in a highly compressed form.
The final task is to shrink these 16 values down into 5 classes. To do this, we first add some more dropout and then a final Dense layer, which has four neurons; one representing each class of gesture. Each of them is connected to all 16 of the outputs from the previous layer. During training, each neuron will learn the combination of previous-layer activations that correspond to the gesture it represents. 
The layer is configured with a "softmax" activation function, which results in the layer’s output being a set of probabilities that sum to 1. This output is what we see in the model’s output tensor.

In [44]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(# input_shape=(batch, 128, 3)
        8, (4, 3),
        padding="same",
        activation="relu",
        input_shape=(128, 3, 1)), #output_shape=(batch, 128, 3, 8)
    tf.keras.layers.MaxPool2D((3, 3)),  # (batch, 42, 1, 8)
    tf.keras.layers.Dropout(0.1),  # (batch, 42, 1, 8)
    tf.keras.layers.Conv2D(16, (4, 1), padding="same", activation="relu", input_shape=(42, 1, 16)),
    tf.keras.layers.MaxPool2D((3, 1), padding="same"),  # (batch, 14, 1, 16)
    tf.keras.layers.Dropout(0.1),  # (batch, 14, 1, 16)
    tf.keras.layers.Flatten(),  # (batch, 224)
    tf.keras.layers.Dense(16, activation="relu"),  # (batch, 16)
    tf.keras.layers.Dropout(0.1),  # (batch, 16)
    tf.keras.layers.Dense(5, activation="softmax")  # (batch, 4)
])

In [ ]:
model.summary()

The model is a classifier with multiple classes. Thus, the fitting includes class weights. In most cases, classes are not perfectly balanced, so this addition takes this into account.

In [45]:
# compile the model
model.compile(optimizer = tf.keras.optimizers.Adam(),
              loss = 'sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [39]:
Y_classes = np.unique(Y_train)
print(Y_classes)

[0 1]


In [46]:
class_weight = {0: 1., 1: 48., 2: 137., 3: 543., 4: 7780.,}

In [ ]:
mean = np.mean(Y_train)
print(mean)

#class_weight = {0: 1.,
#                1: 50.,
#                2: 2.
#                }

In [47]:
# fit - train the model
model.fit(
        X_train, Y_train, epochs=5,
        verbose=1, class_weight=class_weight, 
        #batch_size=128,
        validation_data=(X_val, Y_val),
)

Instructions for updating:
The `validate_indices` argument has no effect. Indices are always validated on CPU and never validated on GPU.
Epoch 1/5
1206/1206 [==============================] - 92s 15ms/step - loss: 1.2250 - accuracy: 0.5711 - val_loss: nan - val_accuracy: 0.8415
Epoch 2/5
1206/1206 [==============================] - 17s 14ms/step - loss: 0.9864 - accuracy: 0.6415 - val_loss: nan - val_accuracy: 0.9022
Epoch 3/5
1206/1206 [==============================] - 17s 14ms/step - loss: 0.9152 - accuracy: 0.6881 - val_loss: nan - val_accuracy: 0.8120
Epoch 4/5
1206/1206 [==============================] - 17s 14ms/step - loss: 0.9298 - accuracy: 0.6834 - val_loss: nan - val_accuracy: 0.8628
Epoch 5/5
1206/1206 [==============================] - 16s 14ms/step - loss: 0.9117 - accuracy: 0.6781 - val_loss: nan - val_accuracy: 0.8658


In [50]:
#evaluate val_dataset before testing on test data. 
model.evaluate(X_val, Y_val)
classifications = model.predict(X_val)
print(classifications[0])
print(Y_val[0])

1296/1296 [==============================] - 6s 4ms/step - loss: nan - accuracy: 0.8658
[0.7329286  0.24515776 0.00422043 0.00630367 0.01138962]
[0]


In [51]:
model.evaluate(X_test, Y_test)
pred = np.argmax(model.predict(X_test), axis=1)
confusion = tf.math.confusion_matrix(labels=tf.constant(Y_test),
                                       predictions=tf.constant(pred),
                                       num_classes=5)
print(confusion)


#print("Loss {}, Accuracy {}".format(loss, acc))
# Convert the model to the TensorFlow Lite format without quantization
#converter = tf.lite.TFLiteConverter.from_keras_model(model)
#tflite_model = converter.convert()

1242/1242 [==============================] - 6s 4ms/step - loss: nan - accuracy: 0.9691
tf.Tensor(
[[38506    28     0     0     0]
 [  918     0     0     0     0]
 [  269     0     0     0     0]
 [   13     0     0     0     0]
 [    0     0     0     0     0]], shape=(5, 5), dtype=int32)


In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
#converter = tf.lite.TFLiteConverter.from_keras_model('CNNmodel.h5')
tflite_model = converter.convert()

In [ ]:
import pathlib
converter = tf.lite.TFLiteConverter.from_keras_model(model)

# These options are for converter optimizaitons
# Try the converter without them and explore model size and accuracy
# Then use them and reconvert the model and explore model
# size an accuracy at that point.

#converter.optimizations = [tf.lite.Optimize.DEFAULT]    # Uncomment this line for Model 2 and Model 3

#def representative_data_gen():                          # Uncomment the following 5 lines for Model 3
#     for input_value, _ in X_test.take(100):
#         yield [input_value]
#converter.representative_dataset = representative_data_gen
#converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]

tflite_model = converter.convert()
#tflite_models_file = pathlib.Path("./tmp/model1.tflite")
tflite_model_file = pathlib.Path("./tmp/model1.tflite")     # Change the filename here for Model2 and Model3!
tflite_model_file.write_bytes(tflite_model)
# Without any optimizations I got
# 20236  (model1.tflite) - the model that was created with 10 epochs
# With the .optimizations property set I got
# 9728 (model2.tflite)


In [ ]:
#Run this cell each time to test your model's accuracy (make sure to change the filename). 
#Tqdm is a Python library used to display smart progress bars that show the progress of Python code execution.
from tqdm import tqdm
# Load TFLite model and allocate tensors.
tflite_model_file = './tmp/model1.tflite'                 # Change the filename here for Model 2 and 3 with optimisations
interpreter = tf.lite.Interpreter(model_path=tflite_model_file)
interpreter.allocate_tensors()

input_index = interpreter.get_input_details()[0]["index"]
output_index = interpreter.get_output_details()[0]["index"]
print(input_index)
print(output_index)

predictions = []

for i in tqdm(range(0,100)):
    data = X_test[i]
    data = data.reshape(1, data.shape[0], data.shape[1], data.shape[2]) 
    #print(data.shape)
    label = Y_test[i]
    interpreter.set_tensor(input_index, data)
    interpreter.invoke()
    predictions.append(interpreter.get_tensor(output_index))
    
#    Y_test_labels.append(label.numpy()[0])
#    X_test.append(X_test)

# For model 1, 
# For model 2, 
# For model 3, 
# Note: since the it/s will depend on the computer on which the model instance is running -- it would vary a bit.

score = 0
for item in range(0,100):
  prediction=np.argmax(predictions[item])
  label = Y_test[item]
  if prediction==label:
    score=score+1

print("Out of 100 predictions I got " + str(score) + " correct")

# Model 1 - 100 Correct
# Model 2 - 99 Correct
# Model 3 - 99 Correct
# Note: training starts from a random intialization the result could be off by 1 or 2 correct.